In [1]:
import tensorflow as tf
import numpy as np
import os
import math
import sys
import pickle
import tarfile
import zipfile
from time import time
from urllib.request import urlretrieve
from sklearn.metrics import confusion_matrix
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [2]:
def print_activations(t):
    print(t.op.name,'',t.get_shape().as_list)   #get_shape获取一个TensorShape对象，然后通过as_list方法返回每一个维度数

In [3]:
def model():
    _IMAGE_SIZE=32
    _IMAGE_CHANNELS=3
    _RESHAPE_SIZE=3*3*128
    _NUM_CLASSES=10

    parameters=[]
    with  tf.name_scope('data'):
        x=tf.placeholder(tf.float32,shape=[None,_IMAGE_SIZE*_IMAGE_SIZE*_IMAGE_CHANNELS],name='images')
        y=tf.placeholder(tf.float32,shape=[None,_NUM_CLASSES],name='Output')
        images=tf.reshape(x,[-1,_IMAGE_SIZE,_IMAGE_SIZE,_IMAGE_CHANNELS],name='images')
        print(images) 
    #conv1
    #这里name_scope实际上是为了解决共享变量的问题，在name_scope下进行tf.Variable(name)
    #如果name重名，会自动检测命名冲突进行处理   
    with tf.name_scope('conv1') as scope:          
        kernel=tf.Variable(tf.truncated_normal([5,5,3,64],dtype=tf.float32,
                                        stddev=1e-1),name='weights')
        #变量解释 [a,b,c,d]分别表示,1表示是否跳过一些样本，比如a=1时，就是从1，2，3...训
        #跳过一些，a=2时就选择1，3，5...，b表示高方向滑动，c表示宽方向滑动，d表示通道滑动
        #same表示当卷积核超出边界时会进行0填充
        conv=tf.nn.conv2d(images,kernel,[1,1,1,1],padding='SAME')
        biases=tf.Variable(tf.constant(0.0,shape=[64],dtype=tf.float32),
                                        trainable=True,name='bias')
        bias=tf.nn.bias_add(conv,biases)
        conv1=tf.nn.relu(bias,name=scope)    #这里返回的是一个tensor（一个张量类），但是这里的name=scope是什么意思？
        print_activations(conv1)
    tf.summary.histogram('Convolution_layers/conv1',conv1)
    tf.summary.scalar('Convolution_layers/conver1',tf.nn.zero_fraction(conv1))

    #这一步时local Response Normalization技术详情可以查看论文中描述
    #lrn1
    with tf.name_scope('lrn1') as scope:
        lrn1=tf.nn.local_response_normalization(conv1,
                                                alpha=1e-4,
                                                beta=0.75,
                                                depth_radius=2,
                                                bias=2.0)
    #pool1
    pool1=tf.nn.max_pool(lrn1,ksize=[1,3,3,1],strides=[1,2,2,1], 
                    padding='VALID',name='pool1')

    print_activations(pool1)

    #conv2
    with tf.name_scope('conv2') as scope:
        kernel = tf.Variable(tf.truncated_normal([5, 5, 64, 64], dtype=tf.float32,
                                                         stddev=1e-1), name='weights')
        conv = tf.nn.conv2d(pool1, kernel, [1, 1, 1, 1], padding='SAME')
        biases = tf.Variable(tf.constant(0.0, shape=[64], dtype=tf.float32),
                                                 trainable=True, name='biases')
        bias = tf.nn.bias_add(conv, biases)
        conv2 = tf.nn.relu(bias, name=scope)
    tf.summary.histogram('Convolution_layers/conv2',conv2)
    tf.summary.scalar('Convolution_layers/conver2',tf.nn.zero_fraction(conv2))
    print_activations(conv2)
    #lrn2
    with tf.name_scope('lrn2') as scope:
        lrn2 = tf.nn.local_response_normalization(conv2,alpha=1e-4,beta=0.75,
                                                depth_radius=2, bias=2.0)
    # pool2
    pool2 = tf.nn.max_pool(lrn2, ksize=[1, 3, 3, 1],strides=[1, 2, 2, 1],
                            padding='VALID',name='pool2')
    print_activations(pool2)

    #conv3
    with tf.name_scope('conv3') as scope:
        kernel =tf.Variable(tf.truncated_normal([3,3,64,128],dtype=tf.float32,
                                                stddev=1e-1),name='weights')
        conv=tf.nn.conv2d(pool2,kernel,[1,1,1,1],padding='SAME')
        biases=tf.Variable(tf.constant(0.0,shape=[128],dtype=tf.float32),
                                        trainable=True,name='biases')
        bias=tf.nn.bias_add(conv,biases)
        conv3=tf.nn.relu(bias,name=scope)
        print_activations(conv3)
    tf.summary.histogram('Convolution_layers/conv3',conv3)
    tf.summary.scalar('Convolution_layers/conver3',tf.nn.zero_fraction(conv3))
    #conv4
    with tf.name_scope('conv4') as scope:
        kernel =tf.Variable(tf.truncated_normal([3,3,128,128],dtype=tf.float32,
                                                stddev=1e-1),name='weights')
        conv=tf.nn.conv2d(conv3,kernel,[1,1,1,1],padding='SAME')
        biases=tf.Variable(tf.constant(0.0,shape=[128],dtype=tf.float32),
                                        trainable=True,name='biases')
        bias=tf.nn.bias_add(conv,biases)
        conv4=tf.nn.relu(bias,name=scope)
        print_activations(conv4)
    tf.summary.histogram('Convolution_layers/conv4',conv4)
    tf.summary.scalar('Convolution_layers/conver4',tf.nn.zero_fraction(conv4))
    #conv5
    with tf.name_scope('conv5') as scope:
        kernel =tf.Variable(tf.truncated_normal([3,3,128,128],dtype=tf.float32,
                                                stddev=1e-1),name='weights')
        conv=tf.nn.conv2d(conv4,kernel,[1,1,1,1],padding='SAME')
        biases=tf.Variable(tf.constant(0.0,shape=[128],dtype=tf.float32),
                                        trainable=True,name='biases')
        bias=tf.nn.bias_add(conv,biases)
        conv5=tf.nn.relu(bias,name=scope)
        print_activations(conv5)
    tf.summary.histogram('Convolution_layers/conv5',conv5)
    tf.summary.scalar('Convolution_layers/conver5',tf.nn.zero_fraction(conv5))

    #pool5
    pool5=tf.nn.max_pool(conv5,ksize=[1,3,3,1],strides=[1,2,2,1],
                            padding='VALID',name='pool5')
    print_activations(pool5)

    #fully_connected1
    with tf.name_scope('fully_connected1') as scope:
        reshape=tf.reshape(pool5,[-1,_RESHAPE_SIZE])
        dim=reshape.get_shape()[1].value
        weights =tf.Variable(tf.truncated_normal([dim,384],dtype=tf.float32,
                                                stddev=1e-1),name='weights')
        print_activations(weights)
        biases=tf.Variable(tf.constant(0.0,shape=[384],dtype=tf.float32),
                                        trainable=True,name='biases')
        local3=tf.nn.relu(tf.matmul(reshape,weights)+biases,name=scope)
        print_activations(local3)
    tf.summary.histogram('Fully connected layers/fc1',local3)
    tf.summary.scalar('Fully connected layers/fc1',tf.nn.zero_fraction(local3))

    #fully_connected2
    with tf.name_scope('fully_connected') as scope:
        weights =tf.Variable(tf.truncated_normal([384,192],dtype=tf.float32,
                                                stddev=1e-1),name='weights')
        print_activations(weights)
        biases=tf.Variable(tf.constant(0.0,shape=[192],dtype=tf.float32),
                                        trainable=True,name='biases')
        local4=tf.nn.relu(tf.matmul(local3,weights)+biases,name=scope)
        print_activations(local4)
    tf.summary.histogram('Fully connected layers/fc2',local4)
    tf.summary.scalar('Fully connected layers/fc4',tf.nn.zero_fraction(local4))

    #output
    with tf.name_scope('output') as scope:
        weights =tf.Variable(tf.truncated_normal([192,_NUM_CLASSES],dtype=tf.float32,
                                                stddev=1e-1),name='weights')
        print_activations(weights)
        biases=tf.Variable(tf.constant(0.0,shape=[_NUM_CLASSES],dtype=tf.float32),
                                        trainable=True,name='biases')
        softmax_linear=tf.add(tf.matmul(local4,weights),biases,name=scope)
    tf.summary.histogram('Fully connected layers/output',softmax_linear)

    global_step=tf.Variable(initial_value=0,name='global_step',trainable=False)
    y_pred_cls=tf.argmax(softmax_linear,axis=1)


    return x,y,softmax_linear,global_step,y_pred_cls

In [4]:
def get_data_set(name="train", cifar=10):
    x = None
    y = None
    l = None

#    maybe_download_and_extract()

    folder_name = "cifar_10" if cifar == 10 else "cifar_100"

    f = open('./data_set/'+folder_name+'/batches.meta', 'rb')
    datadict = pickle.load(f, encoding='latin1')
    f.close()
    l = datadict['label_names']

    if name is "train":
        for i in range(5):
            f = open('./data_set/'+folder_name+'/data_batch_' + str(i + 1), 'rb')
            datadict = pickle.load(f, encoding='latin1')  #提取数据
            f.close()

            _X = datadict["data"]
            _Y = datadict['labels']
            #print('_X')
            #print(_X)
            #print(np.shape(_X))
            _X = np.array(_X, dtype=float) / 255.0
            #print(np.shape(_X))

            _X = _X.reshape([-1, 3, 32, 32])
            #print(np.shape(_X))
            _X = _X.transpose([0, 2, 3, 1])#矩阵转置,里面的编号是指将原来的维度变换到当前维度
                                           #例如，原来的2变换到当前1维度
            #print(np.shape(_X))
            _X = _X.reshape(-1, 32*32*3)
#            print(np.shape(_X))
#            print(np.shape(_X))
#            print(np.shape(_Y))
            if x is None:
                x = _X
                y = _Y
            else:
                x = np.concatenate((x, _X), axis=0)  #将x与读取的_X拼接起来
                y = np.concatenate((y, _Y), axis=0)
#            print(np.shape(x))
#            print(np.shape(y))

    elif name is "test":
        f = open('./data_set/'+folder_name+'/test_batch', 'rb')
        datadict = pickle.load(f, encoding='latin1')
        f.close()

        x = datadict["data"]
        y = np.array(datadict['labels'])

        x = np.array(x, dtype=float) / 255.0
        x = x.reshape([-1, 3, 32, 32])
        x = x.transpose([0, 2, 3, 1])
        x = x.reshape(-1, 32*32*3)

    def dense_to_one_hot(labels_dense, num_classes=10):
        num_labels = labels_dense.shape[0]
        index_offset = np.arange(num_labels) * num_classes
        labels_one_hot = np.zeros((num_labels, num_classes))
        labels_one_hot.flat[index_offset + labels_dense.ravel()] = 1

        return labels_one_hot

    return x, dense_to_one_hot(y), l

In [5]:
def _print_download_progress(count, block_size, total_size):
    pct_complete = float(count * block_size) / total_size
    msg = "\r- Download progress: {0:.1%}".format(pct_complete)
    sys.stdout.write(msg)
    sys.stdout.flush()

In [6]:
def maybe_download_and_extract():
    main_directory = "./data_set/"
    cifar_10_directory = main_directory+"cifar_10/"
    if not os.path.exists(main_directory):
        os.makedirs(main_directory)

        url = "http://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz"
        filename = url.split('/')[-1]
        file_path = os.path.join(main_directory, filename)
        zip_cifar_10 = file_path
        file_path, _ = urlretrieve(url=url, filename=file_path, reporthook=_print_download_progress)

        print()
        print("Download finished. Extracting files.")
        if file_path.endswith(".zip"):
            zipfile.ZipFile(file=file_path, mode="r").extractall(main_directory)
        elif file_path.endswith((".tar.gz", ".tgz")):
            tarfile.open(name=file_path, mode="r:gz").extractall(main_directory)
        print("Done.")

        os.rename(main_directory+"./cifar-10-batches-py", cifar_10_directory)
        os.remove(zip_cifar_10)

In [7]:
get_data_set(name="train")

(array([[0.23137255, 0.24313725, 0.24705882, ..., 0.48235294, 0.36078431,
         0.28235294],
        [0.60392157, 0.69411765, 0.73333333, ..., 0.56078431, 0.52156863,
         0.56470588],
        [1.        , 1.        , 1.        , ..., 0.31372549, 0.3372549 ,
         0.32941176],
        ...,
        [0.1372549 , 0.69803922, 0.92156863, ..., 0.04705882, 0.12156863,
         0.19607843],
        [0.74117647, 0.82745098, 0.94117647, ..., 0.76470588, 0.74509804,
         0.67058824],
        [0.89803922, 0.89803922, 0.9372549 , ..., 0.63921569, 0.63921569,
         0.63137255]]), array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 1.],
        ...,
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.]]), ['airplane',
  'automobile',
  'bird',
  'cat',
  'deer',
  'dog',
  'frog',
  'horse',
  'ship',
  'truck'])

In [8]:
def predict_test(show_confusion_matrix=False):

    i=0
    predicted_class=np.zeros(shape=len(test_x),dtype=np.int)#返回一个新的数组，用零填充
    print('test_x的长度：')
    print(len(test_x))

    while i<len(test_x):
        j=min(i+_BATCH_SIZE,len(test_x))
        batch_xs=test_x[i:j,:]
        #batch_xs是128*3072的大小，最后一个是16*3072
        batch_ys=test_y[i:j,:]
        predicted_class[i:j]=sess.run(y_pred_cls,feed_dict={x:batch_xs,y:batch_ys})
        i=j

    correct=(np.argmax(test_y,axis=1)==predicted_class)
    acc=correct.mean()*100

    correct_numbers=correct.sum()

    print("Accuracy on Test-Set:{0:.2f}%({1}/{2})".format(acc,correct_numbers,len(test_x)))

    if show_confusion_matrix is True:
        cm=confusion_matrix(y_true=np.argmax(test_y,axis=1),y_pred=predicted_class)
        for i in range(_CLASS_SIZE):
            class_name="({}){}".format(i,test_l[i])
            print (cm[i:],class_name)
        class_numbers=["({0})".format(i) for i in range(_CLASS_SIZE)]
        print("".join(class_numbers))
    
    return acc

In [9]:
def train(num_iterations):
    for i in range(num_iterations):
        randidx=np.random.randint(len(train_x),size=_BATCH_SIZE)    #此处返回的是小于冷（train）的离散均匀分布，总共有128个
        batch_xs=train_x[randidx]
        batch_ys=train_y[randidx]

        start_time=time()
        i_global,_=sess.run([global_step,optimizer],feed_dict={x:batch_xs,y:batch_ys})
        duration=time()-start_time


        if(i_global%10==0)or(i==num_iterations-1):
            _loss,batch_acc=sess.run([loss,accuracy],feed_dict={x:batch_xs,y:batch_ys})
            msg= "Glo bal Step: {0:>6}, accuracy: {1:>6.1%}, loss = {2:.2f} ({3:.1f} examples/sec, {4:.2f} sec/batch)"
            print(msg.format(i_global, batch_acc, _loss, _BATCH_SIZE / duration, duration))

            resultmerged=sess.run(merged,feed_dict={x:batch_xs,y:batch_ys})
            train_writer.add_summary(resultmerged,i_global)


        if  (i_global%100==0)or(i==num_iterations-1):
            
            acc=predict_test()

            print('test accuracy is:')
            print(acc)
            saver.save(sess,save_path=_SAVE_PATH,global_step=global_step)
            print("Saved checkpoint")

In [10]:
train_x,train_y,tain_l=get_data_set("train")
test_x,test_y,test_l=get_data_set("test")

x,y,output,global_step,y_pred_cls=model()
_IMG_SIZE = 32
_NUM_CHANNELS = 3
_BATCH_SIZE = 128
_CLASS_SIZE = 10
_ITERATION = 30000
_SAVE_PATH = "tensorboard/cifar-10/"  #先创建好这些文件
_SAVE_BOARD_PATH="tensorboard/board/"

loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=output,labels=y))
optimizer=tf.train.RMSPropOptimizer(learning_rate=1e-3).minimize(loss,global_step=global_step)

correct_prediction=tf.equal(y_pred_cls,tf.argmax(y,axis=1))
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

tf.summary.scalar('loss',loss)
tf.summary.scalar("Accyracy/train",accuracy)
tf.summary.histogram('histogram',accuracy)
saver=tf.train.Saver()
sess=tf.Session()
merged=tf.summary.merge_all()

train_writer=tf.summary.FileWriter(_SAVE_BOARD_PATH,sess.graph)
sess.run(tf.global_variables_initializer())

if _ITERATION!=0:
    train(_ITERATION)

sess.close()

Tensor("data/images_1:0", shape=(?, 32, 32, 3), dtype=float32)
conv1  <bound method TensorShape.as_list of TensorShape([Dimension(None), Dimension(32), Dimension(32), Dimension(64)])>
pool1  <bound method TensorShape.as_list of TensorShape([Dimension(None), Dimension(15), Dimension(15), Dimension(64)])>
conv2  <bound method TensorShape.as_list of TensorShape([Dimension(None), Dimension(15), Dimension(15), Dimension(64)])>
pool2  <bound method TensorShape.as_list of TensorShape([Dimension(None), Dimension(7), Dimension(7), Dimension(64)])>
conv3  <bound method TensorShape.as_list of TensorShape([Dimension(None), Dimension(7), Dimension(7), Dimension(128)])>
conv4  <bound method TensorShape.as_list of TensorShape([Dimension(None), Dimension(7), Dimension(7), Dimension(128)])>
conv5  <bound method TensorShape.as_list of TensorShape([Dimension(None), Dimension(7), Dimension(7), Dimension(128)])>
pool5  <bound method TensorShape.as_list of TensorShape([Dimension(None), Dimension(3), Dimensi

Glo bal Step:    590, accuracy:  56.2%, loss = 1.25 (3667.0 examples/sec, 0.03 sec/batch)
Glo bal Step:    600, accuracy:  52.3%, loss = 1.34 (3774.9 examples/sec, 0.03 sec/batch)
test_x的长度：
10000
Accuracy on Test-Set:47.28%(4728/10000)
test accuracy is:
47.28
Saved checkpoint
Glo bal Step:    610, accuracy:  59.4%, loss = 1.11 (3667.0 examples/sec, 0.03 sec/batch)
Glo bal Step:    620, accuracy:  70.3%, loss = 0.96 (3666.8 examples/sec, 0.03 sec/batch)
Glo bal Step:    630, accuracy:  58.6%, loss = 1.08 (3774.6 examples/sec, 0.03 sec/batch)
Glo bal Step:    640, accuracy:  46.1%, loss = 1.59 (3666.9 examples/sec, 0.03 sec/batch)
Glo bal Step:    650, accuracy:  72.7%, loss = 0.86 (3565.0 examples/sec, 0.04 sec/batch)
Glo bal Step:    660, accuracy:  59.4%, loss = 1.03 (3774.8 examples/sec, 0.03 sec/batch)
Glo bal Step:    670, accuracy:  64.8%, loss = 1.04 (3565.1 examples/sec, 0.04 sec/batch)
Glo bal Step:    680, accuracy:  81.2%, loss = 0.74 (3666.0 examples/sec, 0.03 sec/batch)
Gl

Accuracy on Test-Set:65.66%(6566/10000)
test accuracy is:
65.66
Saved checkpoint
Glo bal Step:   1410, accuracy:  82.8%, loss = 0.57 (3775.0 examples/sec, 0.03 sec/batch)
Glo bal Step:   1420, accuracy:  87.5%, loss = 0.40 (3777.8 examples/sec, 0.03 sec/batch)
Glo bal Step:   1430, accuracy:  78.9%, loss = 0.61 (3670.4 examples/sec, 0.03 sec/batch)
Glo bal Step:   1440, accuracy:  82.0%, loss = 0.51 (3774.8 examples/sec, 0.03 sec/batch)
Glo bal Step:   1450, accuracy:  75.0%, loss = 0.63 (3774.7 examples/sec, 0.03 sec/batch)
Glo bal Step:   1460, accuracy:  79.7%, loss = 0.64 (3669.7 examples/sec, 0.03 sec/batch)
Glo bal Step:   1470, accuracy:  77.3%, loss = 0.60 (3666.8 examples/sec, 0.03 sec/batch)
Glo bal Step:   1480, accuracy:  84.4%, loss = 0.52 (3666.8 examples/sec, 0.03 sec/batch)
Glo bal Step:   1490, accuracy:  78.9%, loss = 0.61 (3666.5 examples/sec, 0.03 sec/batch)
Glo bal Step:   1500, accuracy:  75.0%, loss = 0.74 (3774.8 examples/sec, 0.03 sec/batch)
test_x的长度：
10000
Ac

Glo bal Step:   2230, accuracy:  94.5%, loss = 0.31 (3892.3 examples/sec, 0.03 sec/batch)
Glo bal Step:   2240, accuracy:  79.7%, loss = 0.58 (3777.7 examples/sec, 0.03 sec/batch)
Glo bal Step:   2250, accuracy:  75.8%, loss = 0.66 (3774.8 examples/sec, 0.03 sec/batch)
Glo bal Step:   2260, accuracy:  85.9%, loss = 0.36 (3670.6 examples/sec, 0.03 sec/batch)
Glo bal Step:   2270, accuracy:  82.8%, loss = 0.45 (3669.8 examples/sec, 0.03 sec/batch)
Glo bal Step:   2280, accuracy:  89.8%, loss = 0.34 (3669.8 examples/sec, 0.03 sec/batch)
Glo bal Step:   2290, accuracy:  93.0%, loss = 0.31 (3667.0 examples/sec, 0.03 sec/batch)
Glo bal Step:   2300, accuracy:  90.6%, loss = 0.28 (3774.8 examples/sec, 0.03 sec/batch)
test_x的长度：
10000
Accuracy on Test-Set:68.31%(6831/10000)
test accuracy is:
68.31
Saved checkpoint
Glo bal Step:   2310, accuracy:  90.6%, loss = 0.28 (3777.1 examples/sec, 0.03 sec/batch)
Glo bal Step:   2320, accuracy:  89.8%, loss = 0.29 (3777.8 examples/sec, 0.03 sec/batch)
Gl

Glo bal Step:   3060, accuracy:  90.6%, loss = 0.23 (3889.2 examples/sec, 0.03 sec/batch)
Glo bal Step:   3070, accuracy:  94.5%, loss = 0.28 (3471.4 examples/sec, 0.04 sec/batch)
Glo bal Step:   3080, accuracy:  96.1%, loss = 0.17 (3666.9 examples/sec, 0.03 sec/batch)
Glo bal Step:   3090, accuracy:  93.8%, loss = 0.19 (3565.1 examples/sec, 0.04 sec/batch)
Glo bal Step:   3100, accuracy:  90.6%, loss = 0.28 (3771.5 examples/sec, 0.03 sec/batch)
test_x的长度：
10000
Accuracy on Test-Set:67.66%(6766/10000)
test accuracy is:
67.66
Saved checkpoint
Glo bal Step:   3110, accuracy:  93.8%, loss = 0.22 (3565.2 examples/sec, 0.04 sec/batch)
Glo bal Step:   3120, accuracy:  83.6%, loss = 0.40 (3564.9 examples/sec, 0.04 sec/batch)
Glo bal Step:   3130, accuracy:  89.1%, loss = 0.32 (3667.8 examples/sec, 0.03 sec/batch)
Glo bal Step:   3140, accuracy:  93.8%, loss = 0.18 (3468.7 examples/sec, 0.04 sec/batch)
Glo bal Step:   3150, accuracy:  96.9%, loss = 0.16 (3774.7 examples/sec, 0.03 sec/batch)
Gl

...

Glo bal Step:  26150, accuracy:  96.9%, loss = 0.20 (3777.7 examples/sec, 0.03 sec/batch)
Glo bal Step:  26160, accuracy:  85.9%, loss = 1.34 (3886.5 examples/sec, 0.03 sec/batch)
Glo bal Step:  26170, accuracy: 100.0%, loss = 0.03 (3775.0 examples/sec, 0.03 sec/batch)
Glo bal Step:  26180, accuracy: 100.0%, loss = 0.01 (3889.0 examples/sec, 0.03 sec/batch)
Glo bal Step:  26190, accuracy:  98.4%, loss = 0.03 (3666.9 examples/sec, 0.03 sec/batch)
Glo bal Step:  26200, accuracy:  99.2%, loss = 0.02 (3778.4 examples/sec, 0.03 sec/batch)
test_x的长度：
10000
Accuracy on Test-Set:74.17%(7417/10000)
test accuracy is:
74.17
Saved checkpoint
Glo bal Step:  26210, accuracy:  84.4%, loss = 0.65 (3778.4 examples/sec, 0.03 sec/batch)
Glo bal Step:  26220, accuracy:  98.4%, loss = 0.04 (3777.6 examples/sec, 0.03 sec/batch)
Glo bal Step:  26230, accuracy:  99.2%, loss = 0.02 (3777.4 examples/sec, 0.03 sec/batch)
Glo bal Step:  26240, accuracy:  98.4%, loss = 0.04 (3776.2 examples/sec, 0.03 sec/batch)
Gl

Glo bal Step:  26980, accuracy: 100.0%, loss = 0.01 (3778.4 examples/sec, 0.03 sec/batch)
Glo bal Step:  26990, accuracy:  99.2%, loss = 0.03 (3774.9 examples/sec, 0.03 sec/batch)
Glo bal Step:  27000, accuracy: 100.0%, loss = 0.03 (3772.5 examples/sec, 0.03 sec/batch)
test_x的长度：
10000
Accuracy on Test-Set:73.94%(7394/10000)
test accuracy is:
73.94
Saved checkpoint
Glo bal Step:  27010, accuracy:  97.7%, loss = 0.11 (3774.8 examples/sec, 0.03 sec/batch)
Glo bal Step:  27020, accuracy:  99.2%, loss = 0.11 (3777.7 examples/sec, 0.03 sec/batch)
Glo bal Step:  27030, accuracy:  96.1%, loss = 0.31 (3666.9 examples/sec, 0.03 sec/batch)
Glo bal Step:  27040, accuracy:  98.4%, loss = 0.07 (3667.0 examples/sec, 0.03 sec/batch)
Glo bal Step:  27050, accuracy:  99.2%, loss = 0.05 (3889.1 examples/sec, 0.03 sec/batch)
Glo bal Step:  27060, accuracy: 100.0%, loss = 0.02 (3666.9 examples/sec, 0.03 sec/batch)
Glo bal Step:  27070, accuracy: 100.0%, loss = 0.01 (3776.0 examples/sec, 0.03 sec/batch)
Gl

test_x的长度：
10000
Accuracy on Test-Set:72.00%(7200/10000)
test accuracy is:
72.0
Saved checkpoint
Glo bal Step:  27810, accuracy:  99.2%, loss = 0.05 (3567.9 examples/sec, 0.04 sec/batch)
Glo bal Step:  27820, accuracy:  95.3%, loss = 0.15 (3667.0 examples/sec, 0.03 sec/batch)
Glo bal Step:  27830, accuracy: 100.0%, loss = 0.00 (3889.2 examples/sec, 0.03 sec/batch)
Glo bal Step:  27840, accuracy:  98.4%, loss = 0.04 (3774.8 examples/sec, 0.03 sec/batch)
Glo bal Step:  27850, accuracy: 100.0%, loss = 0.01 (3889.4 examples/sec, 0.03 sec/batch)
Glo bal Step:  27860, accuracy:  99.2%, loss = 0.06 (3667.0 examples/sec, 0.03 sec/batch)
Glo bal Step:  27870, accuracy:  98.4%, loss = 0.16 (3777.0 examples/sec, 0.03 sec/batch)
Glo bal Step:  27880, accuracy:  99.2%, loss = 0.02 (3774.8 examples/sec, 0.03 sec/batch)
Glo bal Step:  27890, accuracy:  97.7%, loss = 0.06 (3774.2 examples/sec, 0.03 sec/batch)
Glo bal Step:  27900, accuracy:  90.6%, loss = 0.23 (3889.1 examples/sec, 0.03 sec/batch)
tes

Glo bal Step:  28630, accuracy:  99.2%, loss = 0.05 (3670.3 examples/sec, 0.03 sec/batch)
Glo bal Step:  28640, accuracy: 100.0%, loss = 0.00 (3771.8 examples/sec, 0.03 sec/batch)
Glo bal Step:  28650, accuracy: 100.0%, loss = 0.01 (3666.9 examples/sec, 0.03 sec/batch)
Glo bal Step:  28660, accuracy:  98.4%, loss = 0.08 (3893.0 examples/sec, 0.03 sec/batch)
Glo bal Step:  28670, accuracy:  99.2%, loss = 0.06 (3773.6 examples/sec, 0.03 sec/batch)
Glo bal Step:  28680, accuracy: 100.0%, loss = 0.00 (3774.8 examples/sec, 0.03 sec/batch)
Glo bal Step:  28690, accuracy: 100.0%, loss = 0.00 (3669.7 examples/sec, 0.03 sec/batch)
Glo bal Step:  28700, accuracy:  99.2%, loss = 0.04 (3774.7 examples/sec, 0.03 sec/batch)
test_x的长度：
10000
Accuracy on Test-Set:73.33%(7333/10000)
test accuracy is:
73.33
Saved checkpoint
Glo bal Step:  28710, accuracy:  98.4%, loss = 0.05 (3666.8 examples/sec, 0.03 sec/batch)
Glo bal Step:  28720, accuracy:  99.2%, loss = 0.03 (3666.9 examples/sec, 0.03 sec/batch)
Gl

Glo bal Step:  29460, accuracy: 100.0%, loss = 0.03 (3668.2 examples/sec, 0.03 sec/batch)
Glo bal Step:  29470, accuracy:  87.5%, loss = 0.45 (3669.8 examples/sec, 0.03 sec/batch)
Glo bal Step:  29480, accuracy:  96.9%, loss = 0.06 (3893.0 examples/sec, 0.03 sec/batch)
Glo bal Step:  29490, accuracy: 100.0%, loss = 0.00 (3889.3 examples/sec, 0.03 sec/batch)
Glo bal Step:  29500, accuracy: 100.0%, loss = 0.01 (3774.8 examples/sec, 0.03 sec/batch)
test_x的长度：
10000
Accuracy on Test-Set:74.45%(7445/10000)
test accuracy is:
74.45
Saved checkpoint
Glo bal Step:  29510, accuracy:  99.2%, loss = 0.03 (3774.9 examples/sec, 0.03 sec/batch)
Glo bal Step:  29520, accuracy:  97.7%, loss = 0.15 (3774.8 examples/sec, 0.03 sec/batch)
Glo bal Step:  29530, accuracy:  97.7%, loss = 0.06 (3774.7 examples/sec, 0.03 sec/batch)
Glo bal Step:  29540, accuracy: 100.0%, loss = 0.01 (3889.1 examples/sec, 0.03 sec/batch)
Glo bal Step:  29550, accuracy:  93.8%, loss = 0.15 (3667.0 examples/sec, 0.03 sec/batch)
Gl

In [10]:
test_x, test_y, test_l = get_data_set("test", cifar=10)
x, y, output, global_step, y_pred_cls = model()
 
_IMG_SIZE = 32
_NUM_CHANNELS = 3
_BATCH_SIZE = 128
_CLASS_SIZE = 10
_SAVE_PATH = "tensorboard/cifar-10/"
 
saver = tf.train.Saver()
sess = tf.Session()
try:
    print("Trying to restore last checkpoint ...")
    last_chk_path = tf.train.latest_checkpoint(checkpoint_dir=_SAVE_PATH)
    saver.restore(sess, save_path=last_chk_path)
    print("Restored checkpoint from:", last_chk_path)
except Exception as e:
    print("Failed to restore checkpoint. Initializing variables instead.")
    sess.run(tf.global_variables_initializer())
    raise e
 
 
i = 0
predicted_class = np.zeros(shape=len(test_x), dtype=np.int)
while i < len(test_x):
    j = min(i + _BATCH_SIZE, len(test_x))
    batch_xs = test_x[i:j, :]
    batch_ys = test_y[i:j, :]
    predicted_class[i:j] = sess.run(y_pred_cls, feed_dict={x: batch_xs, y: batch_ys})
    i = j
 
correct = (np.argmax(test_y, axis=1) == predicted_class)
acc = correct.mean()*100
correct_numbers = correct.sum()
print("Accuracy on Test-Set: {0:.2f}% ({1} / {2})".format(acc, correct_numbers, len(test_x)))
 
cm = confusion_matrix(y_true=np.argmax(test_y, axis=1), y_pred=predicted_class)
for i in range(_CLASS_SIZE):
    class_name = "({}) {}".format(i, test_l[i])
    print(cm[i, :], class_name)
class_numbers = [" ({0})".format(i) for i in range(_CLASS_SIZE)]
print("".join(class_numbers))
 
 
sess.close()

Tensor("data/images_1:0", shape=(?, 32, 32, 3), dtype=float32)
conv1  <bound method TensorShape.as_list of TensorShape([Dimension(None), Dimension(32), Dimension(32), Dimension(64)])>
pool1  <bound method TensorShape.as_list of TensorShape([Dimension(None), Dimension(15), Dimension(15), Dimension(64)])>
conv2  <bound method TensorShape.as_list of TensorShape([Dimension(None), Dimension(15), Dimension(15), Dimension(64)])>
pool2  <bound method TensorShape.as_list of TensorShape([Dimension(None), Dimension(7), Dimension(7), Dimension(64)])>
conv3  <bound method TensorShape.as_list of TensorShape([Dimension(None), Dimension(7), Dimension(7), Dimension(128)])>
conv4  <bound method TensorShape.as_list of TensorShape([Dimension(None), Dimension(7), Dimension(7), Dimension(128)])>
conv5  <bound method TensorShape.as_list of TensorShape([Dimension(None), Dimension(7), Dimension(7), Dimension(128)])>
pool5  <bound method TensorShape.as_list of TensorShape([Dimension(None), Dimension(3), Dimensi